### Configurar advertencias

In [8]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

### Importaciones

In [3]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Para evitar que salgan advertencias de futuras versiones que no nos interesan

## Preparar los datos

### Cargar datos

Vamos a crear una funcion para cargar los datos a un dataframe usando pandas. Un proceso extremadamente sencillo

In [4]:
def load_data(url: str) -> pd.DataFrame | None:
    try:
        return pd.read_excel(url)
    except Exception as err:
        print(err)
        return

Si la funcion logra cargar los datos, seran retornados en un Dataframe, si no, imprime el error y retorna None

### Limpiar datos

Creamos una funcion para limpiar los datos que recibe el dataframe cargado

In [11]:
def cleaning_data(df: pd.DataFrame):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '_')
    subset = ["restaurant_name", "order_time", "delivery_time", "order_hour"]
    df.dropna(subset=subset, inplace=True)
    df.drop_duplicates(inplace=True)
    df["order_time"] = pd.to_datetime(df["order_time"])
    df["delivery_time"] = pd.to_datetime(df["delivery_time"])

Primero se encarga de convertir todas los nombres de las columnas a minusculas y reemplaza sus espacios por ( _ ) para que sea mas facil acceder luego a ellas. Despues verificamos que las filas escenciales con las que vamos a trabajar no sean nulas, de ser asi las elimina, luego eliminamos las filas duplicadas y por ultimo todas las columnas de tipo fecha las transformamos a el tipo de dato correcto en pandas, para compatibilidad y mejor organizacion

### Guardar datos

Este paso consiste en guardar estos datos en una base SQL para su posterior uso (yo usare SQLite3). Esta funcion recibe el dataframe cargado y limpiado y una ruta donde guardar el archivo ".db" donde se almacenara la DB

In [6]:
def save_data(df: pd.DataFrame, db_url: str):
    with sqlite3.connect(db_url) as conn:
        try:
            df.to_sql("pizza_sell_data", conn, if_exists="replace", index=False)
        except Exception as err:
            print(err)
            return

Simplemente intenta conectarse a la base de datos pasada como argumento (que ya debe estar creada) e intenta convertir el dataframe a una tabla SQL, en caso de error pues imprime el error y retorna None

## Generacion de graficos

Vamos a crear una funcion que nos sirva para generar los graficos que necesitemos en todo el proyecto de manera simple

In [9]:
def generate_plot(data: pd.DataFrame, title: str, x: str, y: str, kind: str, theme: str, palette: str, labels: tuple[str, str], path: str = "graphs"):
    try:
        sns.set_theme(theme)
        plt.figure(10, 8)
        if kind == "bar":
            sns.barplot(data, x=x, y=y, palette=palette)
            
        plt.title(title)
        plt.xlabel(labels[0])
        plt.ylabel(labels[1])
        plt.xticks(rotation=90)
        plt.savefig(f"{path}/{title.replace(' ', '-')}.png")
    except Exception as err:
        print(err)
        return

## ETL (Extract, Tranform, Load)

In [12]:
data_url = "Enhanced_pizza_sell_data_2024-25.xlsx"
db_url = "database.db"

df = load_data(data_url)
cleaning_data(df)
save_data(df, db_url)